In [ ]:
# !pip install pandas numpy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ratings = pd.read_csv(r'ml-latest-small\ratings.csv')  # Replace with your path
movies = pd.read_csv(r'ml-latest-small\movies.csv')    # Replace with your path

ratings.head(), movies.head()

In [ ]:
ratings_movies = pd.merge(ratings, movies, on='movieId')

user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')

user_item_matrix = user_item_matrix.fillna(0)

user_item_matrix.head()

In [ ]:
similarity_matrix = cosine_similarity(user_item_matrix)

similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

similarity_df.head()

In [ ]:
# Function to recommend movies for a given user
def recommend_movies(user_id, user_item_matrix, similarity_df, k=5, num_recommendations=10):
    user_ratings = user_item_matrix.loc[user_id]
    rated_movies = user_ratings[user_ratings > 0].index  # Movies already rated by the user

    # Compute weighted average ratings for all unrated movies
    similarity_scores = similarity_df[user_id]  # Similarity scores for the given user
    unrated_movies = user_item_matrix.columns.difference(rated_movies)
    predicted_ratings = {}

    for movie in unrated_movies:
        other_user_ratings = user_item_matrix[movie]
        weighted_sum = np.dot(similarity_scores, other_user_ratings)
        sum_of_similarities = np.sum(similarity_scores[other_user_ratings > 0])

        if sum_of_similarities > 0:
            predicted_ratings[movie] = weighted_sum / sum_of_similarities
        else:
            predicted_ratings[movie] = 0

    # Sort movies by predicted ratings
    recommended_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return [(movies.loc[movies['movieId'] == movie].iloc[0]['title'], rating) for movie, rating in recommended_movies]


In [ ]:
user_id = 1  # Example user
recommendations = recommend_movies(user_id, user_item_matrix, similarity_df, k=5, num_recommendations=10)

print(f"Top movie recommendations for User {user_id}:")
for i, (movie, score) in enumerate(recommendations, 1):
    print(f"{i}. {movie} (Predicted rating: {score:.2f})")